In [ ]:
# Core
import os
import numpy as np
import pandas as pd

# Audio
import librosa

# Deep Learning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Utils
import warnings
warnings.filterwarnings("ignore")


In [ ]:
pip install tgt praatio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 6.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from pathlib import Path

SPLITS_DIR = Path("Data/Splits")

def load_split(file_name, label, split):
    with open(SPLITS_DIR / file_name, "r") as f:
        ids = [line.strip() for line in f if line.strip()]
    return pd.DataFrame({
        "audio_id": ids,
        "label": label,
        "split": split
    })

dfs = []

# Train
dfs.append(load_split("train_laughter_ids.txt", 1, "train"))
dfs.append(load_split("train_negative_ids.txt", 0, "train"))

# Dev
dfs.append(load_split("dev_laughter_ids.txt", 1, "dev"))
dfs.append(load_split("dev_negative_ids.txt", 0, "dev"))

# Test
dfs.append(load_split("test_laughter_ids.txt", 1, "test"))
dfs.append(load_split("test_negative_ids.txt", 0, "test"))

metadata_df = pd.concat(dfs, ignore_index=True)

print("Dataset loaded")
metadata_df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'Data/Splits/train_laughter_ids.txt'

In [ ]:
from pathlib import Path

AUDIO_DIR = Path("Data/Audio")  # confirm this exists

metadata_df["audio_path"] = metadata_df["audio_id"].apply(
    lambda x: AUDIO_DIR / f"{x}.wav"
)

metadata_df.head()


,audio_id,label,split,audio_path
0,7rnL3CUF3ow,1,train,Data/Audio/7rnL3CUF3ow.wav
1,Gi_rfiEtMyk,1,train,Data/Audio/Gi_rfiEtMyk.wav
2,XOtGFXQK9io,1,train,Data/Audio/XOtGFXQK9io.wav
3,w1y9WTzkum8,1,train,Data/Audio/w1y9WTzkum8.wav
4,wGWxW4xEVJQ,1,train,Data/Audio/wGWxW4xEVJQ.wav


In [ ]:
import librosa
import numpy as np

SAMPLE_RATE = 16000
DURATION = 3.0
N_MFCC = 40
MAX_LEN = int(SAMPLE_RATE * DURATION)

def extract_mfcc(audio_path):
    try:
        # Load audio
        y, sr = librosa.load(audio_path, sr=SAMPLE_RATE)

        # Trim or pad to fixed length
        if len(y) > MAX_LEN:
            y = y[:MAX_LEN]
        else:
            y = np.pad(y, (0, MAX_LEN - len(y)))

        # Extract MFCCs
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=N_MFCC)

        # Transpose for LSTM: (time_steps, features)
        return mfcc.T

    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        return None


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

PROJECT_DIR = "/content/drive/MyDrive/Laughter_Project"
DATA_DIR = f"{PROJECT_DIR}/Data"

os.makedirs(DATA_DIR, exist_ok=True)
print("Project directory ready")


Project directory ready
